In [1]:
# Module Imports
import mariadb
# Execption used to stop the Notebook cell execution politely
class StopExecution(Exception):
    def _render_traceback_(self):
        pass
# Connect to the server and return a Connection object for the db_name
#database.
def connectToDB(db_name):
    try:
        return mariadb.connect(
            user="root",
            password="BLue1998",
            host="172.26.117.187", # Use of localhost because 1/ we don't have
#acces to the docker local network 2/ we have made a port redirection from the
#mariadb server to the host
            port=3306,
            database=db_name
        )
    except mariadb.Error as e:
        print(f"Error connecting to MariaDB Platform: {e}")
        raise StopExecution
conn = connectToDB("flight_reservation")

In [2]:
# A bit of dark magic? Just two intrincated query, one to get the TABLES, one
#for each TABLE to get the COLUMNS.
# And a clever use of format to have aligned columns.
def showTablesAndColumns(conn):
    cur = conn.cursor()
    # Query the tables
    cur.execute("SHOW TABLES;")
    # For each table
    for table in cur.fetchall():
        print("**",table[0],"**")
# Defining the output format {:16} indicate at least 16 characters,
#add spaces if necessary
        outputFormat="{:16}\t{:10}\t{}\t{}\t{}\t{}"
# Printing the headers
        print(outputFormat.format("Field","Type","Null","Key","Default","Extra"))
# Query the columns
        cur.execute('SHOW COLUMNS FROM {};'.format(table[0]))
# For each column
        for attribute in cur.fetchall():
# Print the attribute (*attribute give the content of the tuple to
#the format function)
            print(outputFormat.format(*attribute))
showTablesAndColumns(conn)


** Booking **
Field           	Type      	Null	Key	Default	Extra
id_booking      	int(11)   	NO	PRI	None	auto_increment
client_name     	varchar(50)	NO		None	
quantity        	int(2)    	NO		None	
id_flight       	varchar(20)	NO	MUL	None	
** Flight **
Field           	Type      	Null	Key	Default	Extra
id_flight       	varchar(20)	NO	PRI	None	
max_capacity    	int(3)    	NO		None	
current_capacity	int(3)    	NO		None	


## Exercise 5

Create addFlight function.

In [3]:
def addFlight(id_flight, max_capacity):
    conn = connectToDB("flight_reservation")
    cur = conn.cursor()
    try:
        cur.execute("INSERT INTO Flight(id_flight, max_capacity, current_capacity) VALUES (?, ?, ?)", (id_flight, max_capacity, 0))
        conn.commit()
        print("Flight added successfully!")
    except mariadb.Error as e:
        print(f"Error adding flight: {e}")
        conn.rollback()
addFlight('FL003',10)

Error adding flight: Duplicate entry 'FL003' for key 'PRIMARY'


The code below can verify if the Flight has been added successfully.

In [9]:
conn = connectToDB("flight_reservation")
cur = conn.cursor()
cur.execute("SELECT * FROM Flight")
for flight in cur.fetchall():
    print(flight)


('FL001', 10, 0)
('FL002', 10, 0)
('FL003', 10, 0)
('LH6795', 10, 2)


In [5]:
import time
def addBooking(id_flight, client_name, quantity):
    try:
        #Connect to the database.
        conn = connectToDB("flight_reservation")
        cur = conn.cursor()
        cur.execute("SET TRANSACTION ISOLATION LEVEL SERIALIZABLE")
        #Test if the flight is available.
        cur.execute("SELECT current_capacity, max_capacity FROM Flight WHERE id_flight=?", (id_flight,))
        row = cur.fetchone()
        if row is None:
            return False, "Invalid"
        current_capacity, maximum_capacity = row

        #Test if the flight have enough available seats.
        if current_capacity + quantity > maximum_capacity:
            return False, "Full"

        #Wait 3 seconds to simulate payment processing.
        time.sleep(3)

        #Insert a new booking record in the Booking table.
        cur.execute("INSERT INTO Booking (id_flight, client_name, quantity) VALUES (?, ?, ?)", (id_flight, client_name, quantity))
        conn.commit()

        #Update the flight table with the new current capacity.
        cur.execute("UPDATE Flight SET current_capacity=? WHERE id_flight=?", (current_capacity + quantity, id_flight))
        conn.commit()

        return False, "Reserved"
    except mariadb.Error as e:
        print(f"Error adding booking: {e}")
        return True, "Error"

addFlight("LH6795",10)
print(addBooking("LH6795","Client 0",1))
print(addBooking("LH6795","Client 1",4))
print(addBooking("LH6795","Client 2",3))
print(addBooking("LH6795","Client 3",3))

Error adding flight: Duplicate entry 'LH6795' for key 'PRIMARY'
(False, 'Reserved')
(False, 'Reserved')
(False, 'Reserved')
(False, 'Full')


In [6]:
def cleanDB(id_flight):
    conn = connectToDB("flight_reservation")
    cur = conn.cursor()
    cur.execute("DELETE FROM Booking WHERE id_flight=?;",(id_flight,))
    cur.execute("UPDATE Flight SET current_capacity=0 WHERE id_flight=?;",(id_flight,))
    conn.commit() #With out commit the modification are not applied to the database.
cleanDB('LH6795')

## Exercise 6

In [7]:
def processBooking(id_flight,client_name,quantity):
    print("Processing Flight",id_flight,"for",client_name,":",quantity,"seats.")
    retry,status = addBooking(id_flight,client_name,quantity)
    while retry :
        print("Retry Flight",id_flight,"for",client_name,":",quantity,"seats.")
        retry,status = addBooking(id_flight,client_name,quantity)
        print(client_name,":",status)
        print(client_name,":",status,"Should retry:",retry)
# print(processBooking("LH6795","Client 0",1))
# print(processBooking("LH6795","Client 1",4))
# print(processBooking("LH6795","Client 2",3))
# print(processBooking("LH6795","Client 3",3))

In [8]:
import threading
import random
import time

#Test parameters
id_flight="LH6795"
base_name="Client "

#Clean the db for the test
cleanDB(id_flight)

# Creating a lock for seat booking synchronization
booking_lock = threading.Lock()

#Creating several Thread representing several client that try to book simultaneously a random nmber of seat
jobs=[]
for i in range(5):
    jobs.append(threading.Thread(target=processBooking, args=(id_flight, base_name + str(i), random.randint(1, 4))))


#Start each job, giving 0.3s of delay between each
for job in jobs:
    job.start()
    # time.sleep(0.3)
#Wait for all the thread to finish
for job in jobs:
    job.join()
print("Finished")

Processing Flight LH6795 for Client 0 : 4 seats.
Processing Flight LH6795 for Client 1 : 1 seats.
Processing Flight LH6795 for Client 2 : 4 seats.
Processing Flight LH6795 for Client 3 : 2 seats.
Processing Flight LH6795 for Client 4 : 2 seats.
Finished


# Exercise 6.1
There is race concurency and the sit are not filled properly as we can there is only two seat filled instead of 10

In [21]:
conn = connectToDB("flight_reservation")
cur = conn.cursor()
cur.execute("SELECT * FROM Flight")
for flight in cur.fetchall():
    print(flight)
cur.execute("SELECT * FROM Booking")
for booking in cur.fetchall():
    print(booking)

('FL001', 10, 0)
('FL002', 10, 0)
('FL003', 10, 0)
('LH6795', 10, 2)
(231, 'Client 3', 3, 'LH6795')
(232, 'Client 2', 3, 'LH6795')
(233, 'Client 1', 2, 'LH6795')
(234, 'Client 0', 2, 'LH6795')
(235, 'Client 4', 3, 'LH6795')


The result shows that the database takes the last one, if we comment the time.sleep command, it will take the first one.
and for those 2 cases, booking take always all clients.

In [16]:
conn = connectToDB("flight_reservation")
cur = conn.cursor()
cur.execute("SELECT @@tx_ISOLATION;")
result = cur.fetchone()
print(result[0])

REPEATABLE-READ


After added the 'SET TRANSACTION ISOLATION LEVEL SERIALIZABLE', we can see that even time.sleep command has been commented, it takes the last one.

In [17]:
conn = connectToDB("flight_reservation")
cur = conn.cursor()
cur.execute("SET GLOBAL TRANSACTION ISOLATION LEVEL SERIALIZABLE") #加了个global.
cur.execute("SELECT COUNT(*) FROM Flight WHERE id_flight=?",(id_flight,))

In [135]:
conn.close

<bound method Connection.close of <mariadb.connection connected to '172.26.117.187' at 0x7fa9ac05a680>>